In [ ]:
# !pip install pandas

In [6]:
#CLASSIFICAÇÃO FORÇADA DE ITENS NO BANCO DE DADOS

import sqlite3
import pandas as pd
from pathlib import Path

# --- CONFIGURAÇÃO ---
# Copie EXATAMENTE como aparece na tela travada (parte do nome)
TERMO_TRAVADO = "DROGRARIA SAO BRASILIA BR (Total 2x)" 
# A categoria que deveria ter entrado
CATEGORIA_FINAL = "Remédios"  

db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"🔍 Procurando fantasma: '{TERMO_TRAVADO}'...")

# 1. Encontra o item problemático (pendente)
df_ghost = pd.read_sql_query(
    f"SELECT * FROM transactions WHERE description LIKE '%{TERMO_TRAVADO}%' AND category IS NULL", 
    conn
)

if not df_ghost.empty:
    display(df_ghost[['date', 'description', 'amount', 'category']])
    
    confirm = input("Este é o item travado? (S/N): ").upper()
    
    if confirm == 'S':
        # Força a atualização
        cursor.execute(f"""
            UPDATE transactions 
            SET category = '{CATEGORIA_FINAL}', is_manual = 1 
            WHERE description LIKE '%{TERMO_TRAVADO}%' AND category IS NULL
        """)
        conn.commit()
        print(f"✅ Item destravado! Foi classificado como '{CATEGORIA_FINAL}'.")
    else:
        print("Operação cancelada.")
else:
    print("Nenhum item pendente encontrado com esse nome. Pode ser que ele já esteja classificado, mas o Cache do Streamlit esteja enganando.")

conn.close()

🔍 Procurando fantasma: 'DROGRARIA SAO BRASILIA BR (Total 2x)'...
Nenhum item pendente encontrado com esse nome. Pode ser que ele já esteja classificado, mas o Cache do Streamlit esteja enganando.


In [ ]:
# CARREGAMENTO INICIAL
import sqlite3
import pandas as pd
from pathlib import Path

# Caminho do seu banco de dados (confirme se é este mesmo)
db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")

if not db_path.exists():
    print(f"❌ Erro: Banco de dados não encontrado em: {db_path}")
    print("Verifique se o Google Drive está conectado ou ajuste o caminho.")
else:
    print(f"✅ Banco encontrado: {db_path}")
    conn = sqlite3.connect(db_path)

In [10]:
### EXCLUIR TODOS OS LANÇAMENTOS DO BANCO DE DADOS ###

import sqlite3
from pathlib import Path

# --- CONFIGURAÇÃO ---
db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")

print(f"🔌 Conectando em: {db_path}")

try:
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    # 1. Verifica antes
    qtd_antes = cursor.execute("SELECT count(*) FROM transactions").fetchone()[0]
    print(f"📊 Lançamentos ANTES: {qtd_antes}")

    if qtd_antes > 0:
        # 2. AÇÃO: Apagar Tudo
        print("🗑️ Apagando transações...")
        cursor.execute("DELETE FROM transactions")
        
        # 3. SALVAR IMEDIATAMENTE (O Pulo do Gato)
        conn.commit()
        print("💾 Alterações salvas com sucesso!")
        
        # 4. Prova Real
        qtd_depois = cursor.execute("SELECT count(*) FROM transactions").fetchone()[0]
        print(f"📊 Lançamentos DEPOIS: {qtd_depois}")
        
        if qtd_depois == 0:
            print("\n✅ SUCESSO! Banco limpo. Pode reimportar.")
        else:
            print(f"\n❌ ERRO: Ainda restam {qtd_depois} registros.")
            
    else:
        print("✅ O banco já estava vazio.")

    # Verifica se as regras sobreviveram (Segurança)
    try:
        qtd_regras = cursor.execute("SELECT count(*) FROM classification_rules").fetchone()[0]
        print(f"🧠 Regras de Classificação mantidas: {qtd_regras}")
    except:
        print("⚠️ Tabela de regras não encontrada.")

except sqlite3.OperationalError as e:
    print(f"\n❌ ERRO DE ARQUIVO TRAVADO: {e}")
    print("SOLUÇÃO: Feche o navegador do Streamlit e tente de novo.")

finally:
    if 'conn' in locals():
        conn.close()

🔌 Conectando em: G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db
📊 Lançamentos ANTES: 0
✅ O banco já estava vazio.
🧠 Regras de Classificação mantidas: 120


In [11]:
### CONSULTAR CONTEÚDO DO BANCO DE DADOS ###
import sqlite3
import pandas as pd
from pathlib import Path

# Configuração do Caminho (O mesmo que configuramos no sistema)
db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")

if not db_path.exists():
    print(f"❌ Banco não encontrado em: {db_path}")
    print("Verifique se o Google Drive está conectado.")
else:
    conn = sqlite3.connect(db_path)
    print(f"✅ Conectado com sucesso!")

    # 1. Visão Geral (Últimos registros inseridos)
    print("\n--- 🔍 Últimas 10 Transações Inseridas ---")
    query_recent = "SELECT date, description, amount, source, category FROM transactions ORDER BY date DESC LIMIT 10"
    df_recent = pd.read_sql_query(query_recent, conn)
    display(df_recent)

✅ Conectado com sucesso!

--- 🔍 Últimas 10 Transações Inseridas ---


,date,description,amount,source,category


In [5]:
#CORRIGE ERROS DE DIGITAÇÃO

import sqlite3
import pandas as pd
from pathlib import Path

# --- CONFIGURAÇÃO MANUAL (EDITE AQUI) ---
DE_ERRADO = "Academia"     # <--- O nome que você quer eliminar
PARA_CERTO = "Academias"   # <--- O nome correto que deve ficar
# ----------------------------------------

db_path = Path(r"G:\Meu Drive\4. Registros\Glaydson\Orçamento\db\finance_abs.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

print(f"🔧 MODO DIRETO: Trocando '{DE_ERRADO}' por '{PARA_CERTO}'\n")

# 1. Diagnóstico
df_trans = pd.read_sql_query("SELECT * FROM transactions WHERE category = ?", conn, params=(DE_ERRADO,))
df_rules = pd.read_sql_query("SELECT * FROM classification_rules WHERE target_category = ?", conn, params=(DE_ERRADO,))

count_t = len(df_trans)
count_r = len(df_rules)

if count_t == 0 and count_r == 0:
    print(f"⚠️ Nada encontrado com o nome '{DE_ERRADO}'. Verifique maiúsculas/minúsculas.")
else:
    print(f"📊 Encontrados: {count_t} transações e {count_r} regras.")
    
    # 2. Execução Direta
    # Atualiza transações
    if count_t > 0:
        cursor.execute("UPDATE transactions SET category = ? WHERE category = ?", (PARA_CERTO, DE_ERRADO))
        print(f"✅ {cursor.rowcount} transações corrigidas.")
        
    # Atualiza regras
    if count_r > 0:
        cursor.execute("UPDATE classification_rules SET target_category = ? WHERE target_category = ?", (PARA_CERTO, DE_ERRADO))
        print(f"✅ {cursor.rowcount} regras atualizadas.")
        
    conn.commit()
    print("\n🚀 FEITO! Correção aplicada com sucesso.")

conn.close()

🔧 MODO DIRETO: Trocando 'Academia' por 'Academias'

📊 Encontrados: 40 transações e 3 regras.
✅ 40 transações corrigidas.
✅ 3 regras atualizadas.

🚀 FEITO! Correção aplicada com sucesso.
